In [1]:
# your code here
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
import datetime
df = pd.read_csv('/Users/harleyhoffmann/dat-02-22/ClassMaterial/Unit2/data/master.csv', parse_dates=['visit_date', 'calendar_date'])
df['reserve_visitors'] = df['reserve_visitors'].fillna(0)
df

,id,visit_date,visitors,calendar_date,day_of_week,holiday,genre,area,latitude,longitude,reserve_visitors
0,air_ba937bf13d40fb24,2016-01-13,25,2016-01-13,Wednesday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0.0
1,air_ba937bf13d40fb24,2016-01-14,32,2016-01-14,Thursday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0.0
2,air_ba937bf13d40fb24,2016-01-15,29,2016-01-15,Friday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0.0
3,air_ba937bf13d40fb24,2016-01-16,22,2016-01-16,Saturday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0.0
4,air_ba937bf13d40fb24,2016-01-18,6,2016-01-18,Monday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0.0
...,...,...,...,...,...,...,...,...,...,...,...
252103,air_a17f0778617c76e2,2017-04-21,49,2017-04-21,Friday,0,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,6.0
252104,air_a17f0778617c76e2,2017-04-22,60,2017-04-22,Saturday,0,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,37.0
252105,air_a17f0778617c76e2,2017-03-26,69,2017-03-26,Sunday,0,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,35.0
252106,air_a17f0778617c76e2,2017-03-20,31,2017-03-20,Monday,1,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,3.0


In [15]:
# sorting
df.sort_values(by=['id','visit_date'], ascending=True, inplace=True)

train = df.groupby('visit_date').apply(lambda x: x.iloc[:-15])
test = df.groupby('visit_date').apply(lambda x: x.iloc[-15:])

#we're dropping the dates because we moved them into new columns
train.drop('visit_date', axis=1, inplace=True)
test.drop('visit_date', axis=1, inplace=True)

train.drop('calendar_date', axis=1, inplace=True)
test.drop('calendar_date', axis=1, inplace=True)

#we're predicting visitors so that becomes the variable
X_train, y_train = train.drop('visitors', axis=1), train['visitors']
X_test, y_test   = test.drop('visitors', axis=1), test['visitors']

In [11]:
import category_encoders as ce

In [12]:
#using the Target encoder because it had the best score
te = ce.TargetEncoder()
X_train_te = te.fit_transform(X_train, y_train)
X_test_te  = te.transform(X_test, y_test)
#fitting and scoring
gbm = GradientBoostingRegressor()
gbm.fit(X_train_te, y_train)
gbm.score(X_train_te, y_train), gbm.score(X_test_te, y_test)

/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


(0.470152660219043, 0.46643508620732876)

In [5]:
feats = pd.DataFrame({'Feature':X_train_te.columns, 'Importance':gbm.feature_importances_}).sort_values(by='Importance', ascending=False)
feats

,Feature,Importance
0,id,0.876691
1,day_of_week,0.104515
2,holiday,0.007040
7,reserve_visitors,0.004418
6,longitude,0.004402
5,latitude,0.001921
4,area,0.000658
3,genre,0.000355


In [6]:
gbm.get_params()

{'alpha': 0.9,
 'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'ls',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [16]:
ore = ce.OrdinalEncoder()
ohe = ce.OneHotEncoder()

In [17]:
ore.fit_transform(X_train)

id  day_of_week  holiday  genre  area   latitude  \
visit_date                                                             
2016-01-01 43327     1            1        1      1     1  35.735623   
           113306    2            1        1      1     2  33.589216   
           16725     3            1        1      1     3  38.269076   
           116775    4            1        1      1     4  33.319286   
           20816     5            1        1      2     5  34.766093   
...                ...          ...      ...    ...   ...        ...   
2017-04-22 218499  751            2        0      2    13  35.646572   
           8237    752            2        0      1    12  34.386245   
           222776  753            2        0      6    82  35.629564   
           121098  754            2        0      5     6  35.693840   
           82523   755            2        0      4    56  33.581941   

                    longitude  reserve_visitors  
visit_date                                       
2016-01-01 43327   139.651658               0.0  
           113306  130.392813               0.0  
           16725   140.870403               0.0  
           116775  130.508374               0.0  
           20816   135.628100               0.0  
...                       ...               ...  
2017-04-22 218499  139.653247              37.0  
           8237    132.455018              37.0  
           222776  139.684992              37.0  
           121098  139.703549              37.0  
           82523   130.348436              37.0  

[244938 rows x 8 columns]

In [22]:
df.groupby('day_of_week')['visitors'].mean()

day_of_week
Friday       23.072737
Monday       17.177009
Saturday     26.313688
Sunday       23.873362
Thursday     18.922702
Tuesday      17.672137
Wednesday    19.230121
Name: visitors, dtype: float64

In [23]:
te.fit_transform(X_train, y_train)

/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


id  day_of_week  holiday      genre       area  \
visit_date                                                                 
2016-01-01 43327   35.870763    23.058297        1  22.994346  18.325632   
           113306  22.495726    23.058297        1  22.994346  20.673379   
           16725   10.366013    23.058297        1  22.994346  20.465089   
           116775  11.628993    23.058297        1  22.994346  12.699479   
           20816    4.695755    23.058297        1  13.390262   6.170648   
...                      ...          ...      ...        ...        ...   
2017-04-22 218499  10.192308    26.360766        0  13.390262  19.126473   
           8237    37.114286    26.360766        0  22.994346  23.610277   
           222776  15.085714    26.360766        0  22.344232  13.578850   
           121098   6.835938    26.360766        0  18.661331  19.690496   
           82523   10.056338    26.360766        0  22.619626  22.429098   

                    latitude   longitude  reserve_visitors  
visit_date                                                  
2016-01-01 43327   35.735623  139.651658               0.0  
           113306  33.589216  130.392813               0.0  
           16725   38.269076  140.870403               0.0  
           116775  33.319286  130.508374               0.0  
           20816   34.766093  135.628100               0.0  
...                      ...         ...               ...  
2017-04-22 218499  35.646572  139.653247              37.0  
           8237    34.386245  132.455018              37.0  
           222776  35.629564  139.684992              37.0  
           121098  35.693840  139.703549              37.0  
           82523   33.581941  130.348436              37.0  

[244938 rows x 8 columns]

In [29]:
from sklearn.pipeline import make_pipeline

In [30]:
pipe = make_pipeline(te, gbm)
#make a pipeline to do both processes at once

In [31]:
pipe.fit(X_train, y_train)

/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Pipeline(steps=[('targetencoder',
                 TargetEncoder(cols=['id', 'day_of_week', 'genre', 'area'])),
                ('gradientboostingregressor', GradientBoostingRegressor())])

In [32]:
pipe.predict(X_test)

array([43.53941762, 22.20944956, 25.12478599, ..., 28.61792205,
       26.00678461, 28.61792205])

In [33]:
pipe1 = make_pipeline(ce.OrdinalEncoder(), gbm)
pipe2 = make_pipeline(ce.TargetEncoder(), gbm)

In [37]:
pipe1.fit(X_train, y_train)

Pipeline(steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['id', 'day_of_week', 'genre', 'area'],
                                mapping=[{'col': 'id', 'data_type': dtype('O'),
                                          'mapping': air_04341b588bde96cd      1
air_05c325d315cc17f5      2
air_08ba8cd01b3ba010      3
air_09a845d5b5944b01      4
air_1f7f8fa557bc0d55      5
                       ... 
air_cf5ab75a0afb8af9    811
air_1c0b150f9e696a5f    812
air_900d755ebd2f7bbd    813
air_a17f0778617c76e2    814
NaN                      -2
L...
                                          'mapping': Tōkyō-to Nerima-ku Toyotamakita                  1
Fukuoka-ken Fukuoka-shi Daimyō                   2
Miyagi-ken Sendai-shi Kamisugi                   3
Fukuoka-ken Kurume-shi Jōnanmachi                4
Ōsaka-fu Neyagawa-shi Honmachi                   5
                                              ... 
Tōkyō-to Taitō-ku Asakusa                      100
Hyōgo-ken Amagasaki-shi Higashin

In [38]:
pipe1.score(X_test,y_test)

-0.032617380535093554

In [45]:
#This row is the jam
df.drop('calendar_date', axis=1, inplace=True)

In [ ]:
pipe = make_pipeline(ce.TargetEncoder(), gbm)

In [ ]:
df['month'] = pd.DatetimeIndex(df['visit_date']).month
df['year'] = pd.DatetimeIndex(df['visit_date']).year
df['day'] = pd.DatetimeIndex(df['visit_date']).day
df['mov_avg_10'] = df.groupby('id')['visitors'].apply(lambda x: x.rolling(10).mean()).values
#rolling 10 days average ^^

In [ ]:
df['mov_avg_10'] = df['mov_avg_10'].bfill()
#backfilling missing values since the 10 before the previous day would not be there

In [64]:
 #this is an important function for a validation set
def create_val_splits(df, val_units=15, return_val=False):
    """Function that will take in a dataset and split it up into training, validation, and test sets"""
    # split into training, validation, and test sets
    df = df.drop('visit_date', axis=1)
    train = df.groupby('id').apply(lambda x: x.iloc[:-val_units]).reset_index(drop=True)
    test  = df.groupby('id').apply(lambda x: x.iloc[-val_units:]).reset_index(drop=True)
    
    if return_val:
        val   = train.groupby('id').apply(lambda x: x.iloc[-val_units:]).reset_index(drop=True)
        train = train.groupby('id').apply(lambda x: x.iloc[:-val_units]).reset_index(drop=True)
        return train, val, test
    else:
        return train, test

In [71]:
train, val, test = create_val_splits(df, val_units=15, return_val=True)

In [ ]:
#big function should go here

In [72]:
X_train = train.drop('visitors', axis=1)
y_train = train['visitors']

In [73]:
pipe.fit(X_train, y_train)

/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Pipeline(steps=[('targetencoder',
                 TargetEncoder(cols=['id', 'day_of_week', 'genre', 'area'])),
                ('gradientboostingregressor', GradientBoostingRegressor())])

In [74]:
X_val = val.drop('visitors', axis=1)
y_val = val['visitors']

In [75]:
pipe.score(X_val, y_val)

0.6014271300397389

In [76]:
X_val

,id,day_of_week,holiday,genre,area,latitude,longitude,reserve_visitors,month,year,day,mov_avg_10
0,air_00a91d42b08b08d9,Friday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,60.0,3,2017,17,33.0
1,air_00a91d42b08b08d9,Saturday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,41.0,3,2017,18,30.3
2,air_00a91d42b08b08d9,Tuesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,7.0,3,2017,21,30.3
3,air_00a91d42b08b08d9,Wednesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,8.0,3,2017,22,28.8
4,air_00a91d42b08b08d9,Thursday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,11.0,3,2017,23,27.6
...,...,...,...,...,...,...,...,...,...,...,...,...
12420,air_fff68b929994bfbd,Monday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,0.0,4,2017,3,4.8
12421,air_fff68b929994bfbd,Tuesday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,0.0,4,2017,4,4.5
12422,air_fff68b929994bfbd,Wednesday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,2.0,4,2017,5,4.8
12423,air_fff68b929994bfbd,Thursday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,8.0,4,2017,6,5.0
